# Grupos de Investigación e Investigadores
Documento preparado por [Cris Hernández](http://twitter.com/crishernandezco)

> El siguiente documento demuestra el procedimiento realizado en el tratamiento de los datos sobre Centros de Investigación e Investigadores asociados, para todas las regiones en las que PIAS está presente. Se busca dar indicaciones sobre la organización de dichos datos para efectos de incorporarlos de buena forma en el procesamiento de visualización de la información encomendado.

### Procesamiento de datos

In [36]:
import pandas as pd
import requests
import os
import unicodedata
import csv
from collections import defaultdict
import json

In [56]:
'''
Es posible traer los datos desde Google Drive o desde Github, si es que la apertura de dichos datos no es problema.
De otra forma, debiese habilitarse un servidor y una API, o si se utilizará un CMS como Wordpress, alojar los datos
en dicho hosting, utilizando cuentas FTP.
'''

# Ejemplo de datos extraídos desde Github.
sample = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/data/master/ahca-polls/ahca_polls.csv',sep=',')
sample.head(3)

,Start,End,Pollster,Favor,Oppose,Url,Text
0,6/9/17,6/11/17,Public Policy Polling,24,55,http://www.publicpolicypolling.com/pdf/2017/PP...,Do you support or oppose the health care bill ...
1,6/4/17,6/6/17,YouGov,33,45,https://d25d2506sfb94s.cloudfront.net/cumulus_...,Do you approve or disapprove of the American H...
2,5/31/17,6/6/17,Qunnipiac,17,62,https://poll.qu.edu/national/release-detail?Re...,There is a revised Republican health care plan...


## Lectura de datos
Se leen las distintas hojas de los datos entregados en formato EXCEL. Si bien es cierto no reviste mayor
complicación la lectura de datos en dicho formato, es recomendable migrar al ingreso de datos en formatos
que favorecen la lectura entre máquinas (*machine readable*), como CSV, TSV o JSON.

En  este caso se trabajará con las hojas dentro del libro excel, correspondientes a:
- Centros de Investigación, y
- Investigadores

El objetivo final de este proceso es contar con archivo que permite la visualización de dicha información.

In [58]:
# Lectura de datos y creación de nuevos datos en formato CSV.
r = pd.read_excel('centros-investigadores.xlsx', sheetname=0, skiprows=0)
regiones = r.loc[:,['Región','codigo_identidad','Nombre completo ']]

i = pd.read_excel('centros-investigadores.xlsx', sheetname=2, skiprows=2) # investigadores
investigadores = i.loc[:,['id_grupo_investigacion','Nombre']]

c = pd.read_excel('centros-investigadores.xlsx', sheetname=1, skiprows=0) # centros
centros = c.loc[:,['id_grupo_investigacion','Nombre Grupo de Investigación','codigo_identidad']]

investigadores_centros = pd.merge(investigadores, centros, how='left', on=['id_grupo_investigacion'])

reg_inv_centros_merge = pd.merge(regiones, investigadores_centros, how='left', on=['codigo_identidad'])
variables_necesarias = reg_inv_centros_merge.loc[:,['Región','Nombre Grupo de Investigación','Nombre']]
sin_carcteres = variables_necesarias.replace("[^\w\s]",'',regex=True).astype(str)

nuevos_nombres = sin_carcteres.rename(columns={'Región':'region','Nombre Grupo de Investigación':'grupo_investigacion','Nombre':'investigador'})
nuevos_nombres['center'] = 'Grupos de Investigación'

df = nuevos_nombres.loc[:,['center','region','grupo_investigacion','investigador']] 

def remove_accents(row): 
    return unicodedata.normalize('NFKD', row).encode('ASCII', 'ignore').decode('unicode_escape')

df['region'] = df['region'].apply (lambda row: remove_accents(row))
df['region'] = df['region'].apply(lambda x: x.strip())
df['grupo_investigacion'] = df['grupo_investigacion'].apply (lambda row: remove_accents(row))
df['investigador'] = df['investigador'].apply (lambda row: remove_accents(row))

# Muestra de la cabecera de los datos procesados "dendo.csv"
df.to_csv('dendo.csv',sep=',',index=False, quoting=csv.QUOTE_NONE)
df.head()

,center,region,grupo_investigacion,investigador
0,Grupos de Investigación,Tarapaca,Grupo de Alimentos Funcionales y Nutraceuticos...,Leonel Rojo Castillo
1,Grupos de Investigación,Antofagasta,Grupo de Estudios de Productos Naturales y Com...,Gabino Garrido Garrido
2,Grupos de Investigación,Antofagasta,Laboratorio de Quimica Biologica,Glauco Morales
3,Grupos de Investigación,Coquimbo,Valorizacion de especie nativas y tipicas,Angelica Salvatierra Gonzalez
4,Grupos de Investigación,Coquimbo,Reduccion del uso de plaguicidas en frutales d...,Carlos Quiroz Escobar


## Preparando la visualización
Se ha escogido para el *[Tree Layout](http://mbostock.github.io/d3/talk/20111018/tree.html)* para la visualización de las regiones, los grupos de investigación asociados y los investigadores pertenecientes a dichos centros.

Finalmente de transforman los datos CSV en datos estrcuturados de forma jerárquica, los cuales tendrán un formato JSON:

In [35]:
def ctree():
    """ One of the python gems. Making possible to have dynamic tree structure.

    """
    return defaultdict(ctree)


def build_leaf(name, leaf):
    """ Recursive function to build desired custom tree structure

    """
    res = {"name": name}

    # add children node if the leaf actually has any children
    if len(leaf.keys()) > 0:
        res["children"] = [build_leaf(k, v) for k, v in leaf.items()]

    return res


def main():
    tree = ctree()
    with open('dendo.csv') as csvfile:
        reader = csv.reader(csvfile)
        for rid, row in enumerate(reader):
            if rid == 0:
                continue
            leaf = tree[row[0]]
            for cid in range(1, len(row)):
                leaf = leaf[row[cid]]

    res = []
    for name, leaf in tree.items():
        res.append(build_leaf(name, leaf))
    
    with open('dendo.json', 'w') as f:
        json.dump(res[0], f)
    
#     print(json.dumps(res))
    print('Json exported')

main()

Json exported


In [2]:
%run py_d3.py

In [3]:
%%d3

<!DOCTYPE html>
<meta charset="utf-8">
<style type="text/css">

  body {
    padding: 0;
    margin: 0;
  }

  #tree-container {
    border: 1px solid black;
    float: right;
    display: block;
    overflow-x: hidden;
    overflow-y: hidden;
  }

	.node {
    cursor: pointer;
  }

  .overlay{
      background-color:#fff;
  }

  .node circle {
    fill: #fff;
    stroke: steelblue;
    stroke-width: 1.5px;
  }

  .node text {
    font-size:1em;
    font-family:sans-serif;
  }

  .link {
    fill: none;
    stroke: #ccc;
    stroke-width: 1.5px;
  }

  .templink {
    fill: none;
    stroke: red;
    stroke-width: 3px;
  }

  .ghostCircle.show{
      display:block;
  }

  .ghostCircle, .activeDrag .ghostCircle{
       display: none;
  }

</style>
<script src="jquery-1.10.2.min.js"></script>
<script src="d3.v3.min.js"></script>
<script src="dndTree.js"></script>
<body>
    <div id="tree-container"></div>
</body>
</html>
